In [ ]:
! pip install tensorflow_text
! pip install -q tf-models-official

import tensorflow as tf
from tensorflow.keras.preprocessing import text_dataset_from_directory
import os
import shutil
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization

In [2]:
url = 'https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz'

dataset = tf.keras.utils.get_file('aclImdb_v1.tar.gz', url,
                                  untar=True, cache_dir='.',
                                  cache_subdir='')

dataset_dir = os.path.join(os.path.dirname(dataset), 'aclImdb')

train_dir = os.path.join(dataset_dir, 'train')

# remove unused folders to make it easier to load the data
remove_dir = os.path.join(train_dir, 'unsup')
shutil.rmtree(remove_dir)

84131840/84125825 [==============================] - 1s 0us/step


In [3]:
bz = 32
seed = 132
AUTOTUNE = tf.data.AUTOTUNE

raw_train_ds = text_dataset_from_directory(
    '/content/aclImdb/train', 
    batch_size=bz, 
    validation_split=0.2, 
    subset='training', 
    seed=seed
)

raw_train_ds = raw_train_ds.cache().prefetch(buffer_size=AUTOTUNE)

raw_valid_ds = text_dataset_from_directory(
    '/content/aclImdb/train', 
    batch_size=bz, 
    validation_split=0.2, 
    subset='validation', 
    seed=seed
)

raw_valid_ds = raw_valid_ds.cache().prefetch(buffer_size=AUTOTUNE)

raw_test_ds = text_dataset_from_directory('/content/aclImdb/test', batch_size=bz)
raw_test_ds = raw_test_ds.cache().prefetch(buffer_size=AUTOTUNE)



Found 25000 files belonging to 2 classes.
Using 20000 files for training.
Found 25000 files belonging to 2 classes.
Using 5000 files for validation.
Found 25000 files belonging to 2 classes.


In [4]:
# Copied list of bert models

bert_model_name = 'small_bert/bert_en_uncased_L-4_H-512_A-8' 

map_name_to_handle = {
    'bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3',
    'bert_en_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_cased_L-12_H-768_A-12/3',
    'bert_multi_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_multi_cased_L-12_H-768_A-12/3',
    'small_bert/bert_en_uncased_L-2_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-2_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-2_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-2_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-4_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-4_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-4_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-4_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-6_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-6_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-6_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-6_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-8_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-8_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-8_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-8_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-10_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-10_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-10_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-10_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-12_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-12_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-12_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-768_A-12/1',
    'albert_en_base':
        'https://tfhub.dev/tensorflow/albert_en_base/2',
    'electra_small':
        'https://tfhub.dev/google/electra_small/2',
    'electra_base':
        'https://tfhub.dev/google/electra_base/2',
    'experts_pubmed':
        'https://tfhub.dev/google/experts/bert/pubmed/2',
    'experts_wiki_books':
        'https://tfhub.dev/google/experts/bert/wiki_books/2',
    'talking-heads_base':
        'https://tfhub.dev/tensorflow/talkheads_ggelu_bert_en_base/1',
}

map_model_to_preprocess = {
    'bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'bert_en_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_cased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'bert_multi_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_multi_cased_preprocess/3',
    'albert_en_base':
        'https://tfhub.dev/tensorflow/albert_en_preprocess/3',
    'electra_small':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'electra_base':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'experts_pubmed':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'experts_wiki_books':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'talking-heads_base':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
}

tfhub_handle_encoder = map_name_to_handle[bert_model_name]
tfhub_handle_preprocess = map_model_to_preprocess[bert_model_name]

print(f'BERT model selected           : {tfhub_handle_encoder}')
print(f'Preprocess model auto-selected: {tfhub_handle_preprocess}')

BERT model selected           : https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1
Preprocess model auto-selected: https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3


In [5]:
bert_preprocess_layer = hub.KerasLayer(tfhub_handle_preprocess)

In [6]:
sample_output = bert_preprocess_layer(["This is a really dumb movie"])

In [7]:
bert_model_layer = hub.KerasLayer(tfhub_handle_encoder)

In [8]:
bert_model_layer(sample_output)

{'default': <tf.Tensor: shape=(1, 512), dtype=float32, numpy=
 array([[ 9.84100819e-01,  9.78962421e-01,  8.74753743e-02,
          3.59176815e-01, -5.77231497e-02,  8.54240537e-01,
          9.83500421e-01, -9.13034976e-01, -8.19887742e-02,
         -9.92866158e-01, -7.12271631e-02, -9.90375400e-01,
         -2.04509601e-01, -9.99326944e-01,  8.21420848e-01,
          6.64412439e-01,  9.22467634e-02,  2.94853300e-01,
         -2.73620188e-01,  1.16729856e-01,  3.21530074e-01,
         -2.18734056e-01, -9.61498439e-01,  9.57328677e-01,
          8.11854959e-01,  9.93543327e-01, -2.84540385e-01,
         -1.04043067e-01,  3.48465949e-01,  1.06392065e-02,
          1.05839096e-01, -7.29736537e-02,  6.87348604e-01,
         -5.26596427e-01, -4.47368026e-01, -8.97450447e-02,
         -4.25175615e-02, -4.30494070e-01, -7.17469394e-01,
         -4.71287742e-02, -6.28012046e-02,  3.33168097e-02,
          1.21677309e-01, -3.67219061e-01,  7.27900937e-02,
          3.05459380e-01, -9.96024668e

In [19]:
def build_model():
  text_input = tf.keras.layers.Input(shape=(), dtype=tf.string)
  x = hub.KerasLayer(tfhub_handle_preprocess)(text_input)
  x = hub.KerasLayer(tfhub_handle_encoder, True)(x)
  x = x['pooled_output']
  x = tf.keras.layers.Dropout(0.1)(x)
  x = tf.keras.layers.Dense(1, activation="sigmoid")(x)
  model = tf.keras.Model(text_input, x)

  return model


model = build_model()
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None,)]            0                                            
__________________________________________________________________________________________________
keras_layer_4 (KerasLayer)      {'input_word_ids': ( 0           input_2[0][0]                    
__________________________________________________________________________________________________
keras_layer_5 (KerasLayer)      {'encoder_outputs':  28763649    keras_layer_4[0][0]              
                                                                 keras_layer_4[0][1]              
                                                                 keras_layer_4[0][2]              
____________________________________________________________________________________________

In [20]:
epochs = 5
steps_per_epoch = tf.data.experimental.cardinality(raw_train_ds).numpy()
num_train_steps = steps_per_epoch * epochs
num_warmup_steps = int(0.1*num_train_steps)

init_lr = 3e-5
optimizer = optimization.create_optimizer(init_lr=init_lr,
                                          num_train_steps=num_train_steps,
                                          num_warmup_steps=num_warmup_steps,
                                          optimizer_type='adamw')

model.compile(optimizer=optimizer, loss="binary_crossentropy", metrics=["accuracy"])

In [21]:
model.fit(raw_train_ds, epochs=10, validation_data=raw_valid_ds)

Epoch 1/10
625/625 [==============================] - 156s 242ms/step - loss: 0.4814 - accuracy: 0.7570 - val_loss: 0.3886 - val_accuracy: 0.8228
Epoch 2/10
625/625 [==============================] - 150s 240ms/step - loss: 0.3269 - accuracy: 0.8583 - val_loss: 0.3799 - val_accuracy: 0.8412
Epoch 3/10
625/625 [==============================] - 151s 242ms/step - loss: 0.2441 - accuracy: 0.9025 - val_loss: 0.3919 - val_accuracy: 0.8512
Epoch 4/10
625/625 [==============================] - 152s 243ms/step - loss: 0.1894 - accuracy: 0.9276 - val_loss: 0.4347 - val_accuracy: 0.8556
Epoch 5/10
625/625 [==============================] - 150s 240ms/step - loss: 0.1459 - accuracy: 0.9464 - val_loss: 0.4703 - val_accuracy: 0.8580
Epoch 6/10
625/625 [==============================] - 150s 240ms/step - loss: 0.1328 - accuracy: 0.9529 - val_loss: 0.4703 - val_accuracy: 0.8580
Epoch 7/10
625/625 [==============================] - 150s 240ms/step - loss: 0.1332 - accuracy: 0.9513 - val_loss: 0.4703 -

KeyboardInterrupt: ignored

In [22]:
loss, accuracy = model.evaluate(raw_test_ds)

782/782 [==============================] - 83s 106ms/step - loss: 0.4679 - accuracy: 0.8551


In [26]:
pred_examples = ["this movie was so great i burned the cd and broke the tv into a million pieces"]

model.predict(pred_examples)

array([[0.9878491]], dtype=float32)